#영화 추천

### DB 불러오기

In [2]:
from google.colab import drive
from ast import keyword
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd
drive.mount('/content/drive')
df=pd.read_pickle('/content/drive/MyDrive/Project 1/main_pickle_data.pickle')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 사용자에게 정보 받아오기(함수)

In [3]:
# 사용자 검색 영화중 줄거리와 키워드 추출해주는 함수
def movie_story_finder(movie_name):
    if movie_name in df['영화명'].values:
        global user_movie_story
        global user_movie_keyword
        user_movie_story=df[df['영화명']==movie_name]['줄거리_fin']
        user_movie_keyword=df[df['영화명']==movie_name]['키워드']
    return user_movie_story, user_movie_keyword


# 사용자에게 정보 받는 함수
def input_movie_info():

    # 사용자로부터 영화 정보를 입력받습니다.
    global movie_name
    movie_name = input("인상깊은 영화명을 입력하세요 (쉼표로 구분): ").split(',')
    genre = input("좋아하는 장르를 입력하세요 (쉼표로 구분): ").split(',')
    director = input("좋아하는 감독을 입력하세요 (쉼표로 구분): ")
    cast = input("좋아하는 영화배우를 입력하세요 (쉼표로 구분): ").split(',')
    keyword_s=[]
    keyword=[]
    story_s=[]
    story=[]


    # 영화 제목 받아서 키워드, 줄거리 추가
    for i in movie_name:
        movie_story_finder(i)
        keyword_s.extend(user_movie_keyword)
        story_s.extend(user_movie_story)
    for a in keyword_s:
        keyword.extend(a)
    for a in story_s:
        story.extend(a)


    # 입력받은 정보를 딕셔너리에 저장합니다.
    global movie_info
    movie_info = {
            '영화명': movie_name,
            '장르': genre,
            '감독': director,
            '출연': cast,
            '키워드': keyword,
            '줄거리_fin': story
    }

    return movie_info

### 사용자 정보 DB에 추가

In [ ]:
df = df.append(movie_info, ignore_index=True)

### 데이터 전처리(결측치 처리, 품사태깅, 불용어 처리, 벡터화) 및 유사도 측정

In [4]:
tfidf = TfidfVectorizer(stop_words='english')
# 장르
genres_list = df['장르']
flattened_genres = [' '.join(genre_list) for genre_list in genres_list]
tfidf_matrix_g = tfidf.fit_transform(flattened_genres)
# 감독
tfidf_matrix_d = tfidf.fit_transform(df['감독'])
# 출연
actors_list = df['출연']
flattened_actors = [' '.join(actor_list) for actor_list in actors_list]
tfidf_matrix_a = tfidf.fit_transform(flattened_actors)
# 키워드
keywords_list = df['키워드']
flattened_keywords = [' '.join(keyword_list) for keyword_list in keywords_list]
tfidf_matrix_k = tfidf.fit_transform(flattened_keywords)
# 줄거리
stories_list = df['줄거리_fin']
flattened_stories = [' '.join(story_list) for story_list in stories_list]
tfidf_matrix_s = tfidf.fit_transform(flattened_stories)


# 코사인 유사도 측정
cosine_sim_g = cosine_similarity(tfidf_matrix_g, tfidf_matrix_g[-1])
cosine_sim_d = cosine_similarity(tfidf_matrix_d, tfidf_matrix_d[-1])
cosine_sim_a = cosine_similarity(tfidf_matrix_a, tfidf_matrix_a[-1])
cosine_sim_k = cosine_similarity(tfidf_matrix_k, tfidf_matrix_k[-1])
cosine_sim_s = cosine_similarity(tfidf_matrix_s, tfidf_matrix_s[-1])


# 코사인 유사도 데이터 프레임 생성
cos_sim_df = pd.DataFrame(cosine_sim_g, index = df['영화명'],columns=['장르 유사도'] )
cos_sim_df['감독 유사도']=cosine_sim_d
cos_sim_df['출연 유사도']=cosine_sim_a
cos_sim_df['키워드 유사도']=cosine_sim_k
cos_sim_df['줄거리 유사도']=cosine_sim_s
cos_sim_df['평균']=cos_sim_df.mean(axis=1)


# 유사도 높은 순으로 정렬
cos_sim_df = cos_sim_df.transpose()  # Transpose the DataFrame to swap rows and columns
cos_sim_df = cos_sim_df.sort_values(by=['평균'], axis=1, ascending=False)
cos_sim_df = cos_sim_df.transpose()  # Transpose it back to the original orientation
cos_sim_df.head(20)

,장르 유사도,감독 유사도,출연 유사도,키워드 유사도,줄거리 유사도,평균
영화명,,,,,,
Any Man's Death,1.000000,1.0,1.000000,1.000000,0.0,0.800000
북극의 희생자,0.658530,1.0,0.000000,0.000000,0.0,0.331706
아메리칸 룰렛,1.000000,0.0,0.000000,0.601780,0.0,0.320356
포이즌,1.000000,0.0,0.000000,0.601780,0.0,0.320356
다운엔젤,1.000000,0.0,0.000000,0.601780,0.0,0.320356
워치타워,1.000000,0.0,0.000000,0.422717,0.0,0.284543
파이날 캇,1.000000,0.0,0.000000,0.390279,0.0,0.278056
트리거 이펙트,0.783518,0.0,0.000000,0.601780,0.0,0.277060
하드 타임,0.777907,0.0,0.000000,0.601780,0.0,0.275937


# 전체

In [1]:
from google.colab import drive
from ast import keyword
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd
drive.mount('/content/drive')
df=pd.read_pickle('/content/drive/MyDrive/main_pickle_data.pickle')

# 사용자 검색 영화중 줄거리와 키워드 추출해주는 함수
def movie_story_finder(movie_name):
    if movie_name in df['영화명'].values:
        global user_movie_story
        global user_movie_keyword
        user_movie_story=df[df['영화명']==movie_name]['줄거리_fin']
        user_movie_keyword=df[df['영화명']==movie_name]['키워드']
    return user_movie_story, user_movie_keyword


# 사용자에게 정보 받는 함수
def input_movie_info():

    # 사용자로부터 영화 정보를 입력받습니다.
    global movie_name
    movie_name = input("인상깊은 영화명을 입력하세요 (쉼표로 구분): ").split(',')
    genre = input("좋아하는 장르를 입력하세요 (쉼표로 구분): ").split(',')
    director = input("좋아하는 감독을 입력하세요 (쉼표로 구분): ")
    cast = input("좋아하는 영화배우를 입력하세요 (쉼표로 구분): ").split(',')
    keyword_s=[]
    keyword=[]
    story_s=[]
    story=[]


    # 영화 제목 받아서 키워드, 줄거리 추가
    for i in movie_name:
        movie_story_finder(i)
        keyword_s.extend(user_movie_keyword)
        story_s.extend(user_movie_story)
    for a in keyword_s:
        keyword.extend(a)
    for a in story_s:
        story.extend(a)


    # 입력받은 정보를 딕셔너리에 저장합니다.
    movie_info = {
            '영화명': movie_name,
            '장르': genre,
            '감독': director,
            '출연': cast,
            '키워드': keyword,
            '줄거리_fin': story
    }
    global df
    df = df.append(movie_info, ignore_index=True)

    for i in movie_name:
        for a in df['영화명']:
            if i==a:
                df=df.drop(df[df['영화명']==i].index,axis=0)

    tfidf = TfidfVectorizer(stop_words='english')
    # 장르
    genres_list = df['장르']
    flattened_genres = [' '.join(genre_list) for genre_list in genres_list]
    tfidf_matrix_g = tfidf.fit_transform(flattened_genres)
    # 감독
    tfidf_matrix_d = tfidf.fit_transform(df['감독'])
    # 출연
    actors_list = df['출연']
    flattened_actors = [' '.join(actor_list) for actor_list in actors_list]
    tfidf_matrix_a = tfidf.fit_transform(flattened_actors)
    # 키워드
    keywords_list = df['키워드']
    flattened_keywords = [' '.join(keyword_list) for keyword_list in keywords_list]
    tfidf_matrix_k = tfidf.fit_transform(flattened_keywords)
    # 줄거리
    stories_list = df['줄거리_fin']
    flattened_stories = [' '.join(story_list) for story_list in stories_list]
    tfidf_matrix_s = tfidf.fit_transform(flattened_stories)


    # 코사인 유사도 측정
    cosine_sim_g = cosine_similarity(tfidf_matrix_g, tfidf_matrix_g[-1])
    cosine_sim_d = cosine_similarity(tfidf_matrix_d, tfidf_matrix_d[-1])
    cosine_sim_a = cosine_similarity(tfidf_matrix_a, tfidf_matrix_a[-1])
    cosine_sim_k = cosine_similarity(tfidf_matrix_k, tfidf_matrix_k[-1])
    cosine_sim_s = cosine_similarity(tfidf_matrix_s, tfidf_matrix_s[-1])


    # 코사인 유사도 데이터 프레임 생성
    cos_sim_df = pd.DataFrame(cosine_sim_g, index = df['영화명'],columns=['장르 유사도'] )
    cos_sim_df['감독 유사도']=cosine_sim_d
    cos_sim_df['출연 유사도']=cosine_sim_a
    cos_sim_df['키워드 유사도']=cosine_sim_k
    cos_sim_df['줄거리 유사도']=cosine_sim_s
    cos_sim_df['평균']=cos_sim_df.mean(axis=1)


    # 유사도 높은 순으로 정렬
    cos_sim_df = cos_sim_df.transpose()  # Transpose the DataFrame to swap rows and columns
    cos_sim_df = cos_sim_df.sort_values(by=['평균'], axis=1, ascending=False)
    cos_sim_df = cos_sim_df.transpose()  # Transpose it back to the original orientation

    return cos_sim_df.head(20)

Mounted at /content/drive


In [2]:
input_movie_info()

인상깊은 영화명을 입력하세요 (쉼표로 구분): 어벤져스,살인의추억
좋아하는 장르를 입력하세요 (쉼표로 구분): 로맨스
좋아하는 감독을 입력하세요 (쉼표로 구분): 봉준호
좋아하는 영화배우를 입력하세요 (쉼표로 구분): 이병헌


<ipython-input-1-67e67aa8a1c9>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(movie_info, ignore_index=True)


,장르 유사도,감독 유사도,출연 유사도,키워드 유사도,줄거리 유사도,평균
영화명,,,,,,
"[어벤져스, 살인의추억]",1.000000,1.0,1.000000,1.000000,1.000000,1.000000
세리가 돌아왔다,0.707076,0.0,0.586032,0.000000,0.000000,0.258622
내일은 사랑,0.707076,0.0,0.523314,0.000000,0.000000,0.246078
설국열차,0.000000,1.0,0.000000,0.041655,0.026543,0.213640
옥자,0.000000,1.0,0.000000,0.000000,0.039149,0.207830
기생충,0.000000,1.0,0.000000,0.000000,0.007457,0.201491
캡틴 아메리카: 윈터 솔져,0.000000,0.0,0.000000,0.618416,0.385564,0.200796
마더,0.000000,1.0,0.000000,0.000000,0.003232,0.200646
인플루엔자,0.000000,1.0,0.000000,0.000000,0.002689,0.200538
